In [1]:
# DO NOT DELETE THIS DO NOT DELETE THIS DO NOT DELETE THIS 
from tkinter import *
from PIL import ImageTk, Image
%run ./base_exercises.ipynb

# Global variable to keep track of the current exercise index
current_index = 0

# List to store instances of `BaseExercise` class
exercises = []

# Create a Tkinter window
window = Tk()
window.title("Exercise Details")
icon = PhotoImage(file='../Shared Components/Assets/workout_icon.png')
window.iconphoto(True, icon)

# Function to display exercise details in the window
def display_exercise_details():
    global current_index 
    
    # Get the current exercise
    exercise = exercises[current_index]
    
    # Clear previous labels and text
    for widget in window.winfo_children():
        widget.destroy()

    # Create labels to display exercise details
    Label(window, text="Name:").grid(row=0, column=0, sticky="w")
    Label(window, text=exercise.name).grid(row=0, column=1, sticky="w")

    Label(window, text="Description:").grid(row=1, column=0, sticky="w")
    Label(window, text=exercise.description).grid(row=1, column=1, sticky="w")

    Label(window, text="Difficulty:").grid(row=2, column=0, sticky="w")
    Label(window, text=exercise.difficulty).grid(row=2, column=1, sticky="w")

    Label(window, text="Instructions:").grid(row=3, column=0, sticky="w")
    instructions_text = Text(window, height=6, width=40)
    instructions_text.insert(END, exercise.instructions)
    instructions_text.grid(row=3, column=1, sticky="w")
    
    # Load and display the image
    img = Image.open(exercise.image_path)
    img = img.resize((100, 100), Image.LANCZOS)  # Use LANCZOS for resizing
    photo_img = ImageTk.PhotoImage(img)
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img  # Keep a reference to avoid garbage collection
    img_label.grid(row=0, column=2, rowspan=4)   

    # Create navigation buttons
    prev_button = Button(window, text="Previous", command=show_previous_exercise)
    prev_button.grid(row=4, column=0, sticky="w")

    next_button = Button(window, text="Next", command=show_next_exercise)
    next_button.grid(row=4, column=1, sticky="e")

# Function to show the previous exercise
def show_previous_exercise():
    global current_index
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show the next exercise
def show_next_exercise():
    global current_index
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()

# Create some exercise instances
push_up = BaseExercise(
    "Push-Ups",
    "Upper Body Strengthening Movement",
    30,
    "medium",
    ["chest", "triceps", "shoulders"],
    "1. Place your hands on the floor.\n2. Keep your body straight.\n3. Bend your elbows to lower your chest towards the ground.\n4. Then push back up",
    "../Shared Components/Assets/push-up.png"  # Path to the image file
)

squats = BaseExercise(
    "Squats",
    "Lower Body Strengthening Movement",
    30,
    "medium",
    ["quadriceps", "hamstrings", "glutes"],
    "1. Stand with your feet shoulder-width apart.\n2. Lower your body by bending your knees and hips.\n3. Keep your chest upright and your back straight.\n4. Return to the starting position by pushing through your heels.",
    "../Shared Components/Assets/squat.png"  # Path to the image file
)

# Add exercises to the list
exercises.extend([push_up, squats])

# Display the first exercise
display_exercise_details()

# Run the Tkinter event loop
window.mainloop()


In [3]:
from tkinter import *
from PIL import ImageTk, Image
%run ./base_exercises.ipynb

# Global variable to keep track of the current exercise index
current_index = 0

# List to store instances of 'BaseExercise' class
exercises = []

# Create a Tkinter window
window = Tk()
window.title("Exercise Details")
icon = PhotoImage(file='../Shared Components/Assets/workout_icon.png')
window.iconphoto(True, icon)

# Function to display exercise details in the window
def display_exercise_details():
    # Access global var to get current index
    global current_index
    
    # Get the current exercise
    exercise = exercises[current_index]
    
    # Clears previous labels & texts
    for widget in window.winfo_children():
        widget.destroy()
        
    # Create labels to display exercise details
    # Name Label
    Label(window, text="Name:").grid(row=0, column=0, sticky="w")
    Label(window, text=exercise.name).grid(row=0, column=1, sticky="w")

    # Description Label
    Label(window, text="Description:").grid(row=1, column=0, sticky="w")
    Label(window, text=exercise.description).grid(row=1, column=1, sticky="w")
    
    # Duration Label
    Label(window, text="Duration:").grid(row=2, column=0, sticky="w")
    Label(window, text=str(exercise.duration)+ " seconds").grid(row=2, column=1, sticky="w")
    
    # Difficulty Label
    Label(window, text="Difficulty:").grid(row=3, column=0, sticky="w")
    Label(window, text=exercise.difficulty).grid(row=3, column=1, sticky="w")
    
    # Muscle Groups Label
    Label(window, text="Muscles Target:").grid(row=4, column=0, sticky="w")
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    Label(window, text=muscle_groups_str).grid(row=4, column=1, sticky="w")
    
    # Instructions Label
    Label(window, text="Instructions:").grid(row=5, column=0, sticky="w")
    instructions_text = Text(window, height=6, width=40)
    instructions_text.insert(END, exercise.instructions)
    instructions_text.grid(row=5, column=1, sticky="w")
    

# Create some exercise instances
push_up = BaseExercise(
    "Push-Ups",
    "Upper Body Strengthening Movement",
    30,
    "medium",
    ["Chest", "Triceps", "Shoulders"],
    "1. Place your hands on the floor.\n2. Keep your body straight.\n3. Bend your elbows to lower your chest towards the ground.\n4. Then push back up",
    "../Shared Components/Assets/push-up.png"  # Path to the image file
)

squats = BaseExercise(
    "Squats",
    "Lower Body Strengthening Movement",
    30,
    "medium",
    ["Quadriceps", "Hamstrings", "Glutes"],
    "1. Stand with your feet shoulder-width apart.\n2. Lower your body by bending your knees and hips.\n3. Keep your chest upright and your back straight.\n4. Return to the starting position by pushing through your heels.",
    "../Shared Components/Assets/squat.png"  # Path to the image file
)

# Adding exercises to the list
exercises.extend([push_up, squats])

# Displaying the first exercise
display_exercise_details()

window.mainloop()